<center> <h1> UGCA Assignment 3 </h1> </center>
<center> <h3> Matt Viteri, Colin Chu, Darasimi Oluwaniyi, Vikram Gupta, & Yeggi Lee </h3> </center>

<b> High level description: </b> The objective of this group assignment is to create the building blocks of a crowdsourced recommendation system. This recommendation system should accept user inputs about desired attributes of a product and come up with 3 recommendations. In this case, we used beer recommendations from the following website: https://www.beeradvocate.com/beer/top-rated/

In [1]:
import numpy as np
import pandas as pd
import re
import textblob
import nltk
import matplotlib.pyplot as plt
import random

from sklearn import manifold
from configparser import ConfigParser
from IPython.display import display
from collections import Counter
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

stop_words = set(stopwords.words('english'))

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/stevenoluwaniyi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### TO KEEP IN MIND: 

Your submission should include the following: 
1. Names of all team members inside the document (only one submission per team)
2. All scripts 
3. The average sentiment and similarity scores for the three products you recommended in task E.
4. Your analyses and answer to task F. Make sure you show the ratings, similarity scores and sentiments for the products you recommend in tasks E and F. 

#### Guidelines
1.    For similarity analysis, use spacy (instead of simple cosine similarity). The script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name, product_review and similarity_score. 
2.    The same beer may appear on multiple rows (in the 300 rows with highest similarity scores) since there are multiple reviews for each beer. Reduce the number of rows by averaging the user_rating of each beer (can do it in Excel). After this operation, each beer will be present only in one row. The user_rating column will now have average ratings for each beer. 
3.    For task F, you do the same averaging, but with ALL rows of data (that is, as many rows as there are reviews).   

### Task A
Extract about 6k reviews of beers from Beeradvocate.com. It is a single-page listing of 250 top-rated beers (avoids the pagination feature, which you need in cases where listings go on for many pages). Scrape 25 reviews and well as the ratings provided by users for each beer. The way beeradvocate.com organizes reviews is that it provides 25 reviews per page, which means you can avoid the pagination part for the reviews as well. The CSV output file should have 3 columns: product_name, product_review, and user_rating. 
- The scrapper is in the file, "scraper.py" and the output is in "output.csv"
- Approximately ____k reviews were scraped

In [2]:
orig_df = pd.read_csv('output.csv')
orig_df.head(10)

,beer,look,smell,taste,feel,overall,comment
0,Kentucky Brunch Brand Stout,4.75,4.75,4.75,4.75,5.00,"2016 Silver Wax. Aroma has whiskey, maple, tof..."
1,Kentucky Brunch Brand Stout,4.50,4.75,4.75,4.75,4.75,The beer pours Pitch Black with a frothy tan h...
2,Kentucky Brunch Brand Stout,4.75,4.75,4.50,5.00,4.75,Probably the smoothest beer I have ever had. S...
3,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,"Dark black, very thick, a little bit of tan he..."
4,Kentucky Brunch Brand Stout,4.50,5.00,5.00,5.00,5.00,Poured black as ink with thin ruby edges at 58...
5,Kentucky Brunch Brand Stout,5.00,5.00,4.50,5.00,4.75,she’s got heat....but man is she somethin nice...
6,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,Amazing brew. The maple aroma pours out of thi...
7,Kentucky Brunch Brand Stout,5.00,5.00,3.75,4.50,4.25,Finally got to try the white whale. Pours like...
8,Kentucky Brunch Brand Stout,5.00,5.00,4.75,5.00,4.75,Had this on tap at the 12-15-18 release in Dec...
9,Kentucky Brunch Brand Stout,4.75,5.00,4.75,5.00,5.00,Had this on tap At the KBBS release 12/15/18 p...


### Task B
Assume that a customer, who will be using this recommendation system, has specified 3 attributes in a beer. One website describes multiple attributes of beer: https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/
- <b> Aggressive: </b> Boldly assertive aroma and/or taste
- <b>Balanced:</b> Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish
- <b>Complex:</b> Multidimensional; many flavors and sensations on the palate
- <b>Crisp: </b>Highly carbonated; effervescent
- <b>Fruity:</b> Flavors reminiscent of various fruits or <b>Hoppy: </b> Herbal, earthy, spicy, or citric aromas and flavors of hops or <b>Malty</b>: Grainy, caramel-like; can be sweet or dry
- <b>Robust:</b> Rich and full-bodied

Choose three attributes from the above list. If you choose another product (like electric guitars), choose a different set – e.g., action, neck, pickup (for guitars). It is a good idea to perform a word frequency analysis of the data you have collected to make sure that the above attributes are actually mentioned.   

NOTE: Please don't rerun; takes a long time

TODO: is strong the same thing as aggresive?

In [3]:
top_50_beers = orig_df.groupby('beer').count()['comment'].sort_values(ascending=False).index[:50].tolist()
df = orig_df[orig_df['beer'].isin(top_50_beers)].groupby('beer').head(200)

In [4]:
same_map = {
    'aggresive': ['bold', 'assertive'],
    'balanced': ['balance', 'multidimensional'],
    'complex': ['complexing'],
    'crisp': ['crispy', 'carbonated', 'carbonation'],
    'fruity': ['fruit', 'tart', 'apple', 'fig', 'orange', 'apples', 'oranges', 'raspberry', 'raspberries', 
               'cherry', 'grapefruit', 'tropical'],
    'hoppy': ['herbal', 'earthy', 'spicy', 'citric', 'hops', 'hop', 'citrus'],
    'malty': ['malt', 'grainy', 'caramel', 'maple'],
    'robust': ['rich', 'full-bodied']
}

same_inv_map = {word: root for root, words in same_map.items() for word in words}

def splitter(data):
    words = re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", data.lower()).split()
    return [textblob.Word(word).lemmatize() for word in words]

def convert_same(text):
    return [(same_inv_map[word] if word in same_inv_map else word) for word in splitter(text)]

def wordCount(data):
    filtered = [w for w in convert_same(' '.join(data['comment'])) if w not in stop_words]
    return Counter(filtered).most_common(100)

df['split_comments'] = df['comment'].map(convert_same)

wordCount(df)

[('fruity', 12990),
 ('beer', 10816),
 ('hoppy', 7737),
 ('malty', 6906),
 ('head', 6702),
 ('taste', 6433),
 ('dark', 5559),
 ('flavor', 5285),
 ('wa', 4741),
 ('crisp', 4470),
 ('like', 4299),
 ('chocolate', 4235),
 ('one', 4230),
 ('aroma', 3878),
 ('nice', 3847),
 ('sweet', 3779),
 ('smell', 3714),
 ('bourbon', 3714),
 ('note', 3648),
 ('good', 3572),
 ('well', 3406),
 ('finish', 3381),
 ('bottle', 3297),
 ('vanilla', 3283),
 ('light', 3158),
 ('pours', 2859),
 ('bit', 2840),
 ('nose', 2767),
 ('smooth', 2723),
 ('glass', 2710),
 ('balanced', 2682),
 ('great', 2681),
 ('overall', 2649),
 ('little', 2647),
 ('brown', 2608),
 ('really', 2590),
 ('feel', 2442),
 ('lacing', 2430),
 ('body', 2422),
 ('medium', 2364),
 ('coffee', 2364),
 ('mouthfeel', 2353),
 ('ha', 2298),
 ('alcohol', 2293),
 ('black', 2233),
 ('color', 2230),
 ('get', 2166),
 ('white', 2161),
 ('poured', 2091),
 ('much', 2066),
 ('sour', 2056),
 ('oak', 2007),
 ('stout', 1986),
 ('lot', 1962),
 ('thick', 1916),
 ('bitt

#### Results
The most common attributes out of the list above are: 
- Fruity
- Hoppy
- Malty 
- Crisp
- Balanced
- Robust
- Complex

### Task C 
Perform a similarity analysis with the 3-attribute set and the reviews. See guidelines below on how to do this. From the output file, choose 300 reviews that have the highest similarity scores with the attribute set you have chosen. 

Run the following line in your terminal AFTER you install spacy
<br />
`python -m spacy download en_core_web_lg`

In [5]:
import spacy

nlp = spacy.load("en_core_web_lg")
STOPWORDS = nlp.Defaults.stop_words

In [6]:
attributes = ['crisp', 'balanced', 'fruity']
attstr = ' '.join(attributes)

In [7]:
# ignore stopwords, punctuation and pronouns
def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return ' '.join(result)

In [8]:
def calculate_similarity(text):
    base = nlp(attstr)
    compare = nlp(process_text(text))
    return base.similarity(compare)

In [9]:
sims = []

for i, r in df.iterrows():
    s = calculate_similarity(r.comment)
    d = {
        'product_name': r.beer,
        'product_review': r.comment,
        'user_rating': r.overall,
        'similarity_score': s
    }
    sims.append(d)

/Users/stevenoluwaniyi/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/Users/stevenoluwaniyi/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)


In [10]:
sims_df = pd.DataFrame(sims)
sims_df.sort_values(by=['similarity_score'], ascending=False, inplace=True)
sims_df[0:300]

,product_name,product_review,similarity_score,user_rating
8163,Zombie Dust,"Color is a perfect golden ale color, pleasant ...",0.839829,4.75
9211,Pliny The Elder,Pours a clear vibrant marigold colour. Citrusy...,0.837084,4.75
1468,Nelson,"Pours a very light, pale yellow color, with a ...",0.835595,4.50
4213,Pseudo Sue,Murky creamsicle orange in color. Poured with ...,0.833806,4.50
4561,Pseudo Sue,"Pours a hazy orange with soft, thcíck foam. S...",0.833091,4.50
281,Green,12/2014 - this is super fresh and tasty! Pinea...,0.833072,4.50
7852,Zombie Dust,Nice amber color with a white head that dedica...,0.832043,4.25
2120,Temptation,Review from notes. Golden color with white hea...,0.831767,4.25
9959,Heady Topper,Slightly hazy yellow color with two finger hea...,0.831209,4.50
4423,Abrasive Ale,"Pours a golden and clear, one finger head. Sme...",0.829789,4.50


In [11]:
sims_df.to_csv('similarity_score.csv', index=None)

### Task D. 
Perform sentiment analysis on these 300 reviews and sort them (high to low) by the sentiment scores.

In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

In [13]:
df_sims_sentiment = pd.read_csv('similarity_score.csv')
df_sims_sentiment['Sentiment Score'] = df_sims_sentiment['product_review'].map(lambda x: sentiment_analyzer_scores(x))

In [14]:
df_sims_sentiment.sort_values(by=['Sentiment Score'], ascending=False, inplace=True)
print(df_sims_sentiment.head())
df_sims_sentiment.to_csv('similarity_score&sentiment_score.csv', index=None)

                               product_name  \
3602                             Fou' Foune   
2514                                  Citra   
5534  Fourth Dementia - Bourbon Barrel-Aged   
4233      Bourbon Barrel Aged Plead The 5th   
8826                              The Abyss   

                                         product_review  similarity_score  \
3602  Review (11 months of age)- 75cl bottle from Au...          0.729853   
2514  Finally had a chance to try this - perhaps the...          0.744581   
5534  I got a bottle of Kuhnhenn's Bourbon Barrel Fo...          0.702732   
4233  A big thanks to jdhowe for sharing this recent...          0.721373   
8826  Deschutes Brewery "The Abyss" 22 fl. oz. brown...          0.592158   

      user_rating  Sentiment Score  
3602         4.75           0.9998  
2514         5.00           0.9995  
5534         4.75           0.9994  
4233         4.50           0.9994  
8826         4.50           0.9994  


In [15]:
df_sentiment_300 = pd.read_csv('similarity_score.csv')
df_sentiment_300 = df_sentiment_300[0:300]
print(df_sentiment_300.shape)
df_sentiment_300.head()

(300, 4)


,product_name,product_review,similarity_score,user_rating
0,Zombie Dust,"Color is a perfect golden ale color, pleasant ...",0.839829,4.75
1,Pliny The Elder,Pours a clear vibrant marigold colour. Citrusy...,0.837084,4.75
2,Nelson,"Pours a very light, pale yellow color, with a ...",0.835595,4.50
3,Pseudo Sue,Murky creamsicle orange in color. Poured with ...,0.833806,4.50
4,Pseudo Sue,"Pours a hazy orange with soft, thcíck foam. S...",0.833091,4.50


In [16]:
df_sentiment_300['Sentiment Score'] = df_sentiment_300['product_review'].map(lambda x: sentiment_analyzer_scores(x))
df_sentiment_300.sort_values(by=['Sentiment Score'], ascending=False, inplace=True)
df_sentiment_300.head()

,product_name,product_review,similarity_score,user_rating,Sentiment Score
162,Abt 12,L- Pours a two and a half finger rocky headed ...,0.801703,4.75,0.9972
63,Green,"16 ounce can into tulip glass, canned on 3/8/2...",0.813388,4.50,0.9970
147,Fou' Foune,"750ml, bottled august 2012 A- Ligh orange col...",0.802747,4.25,0.9964
68,Nelson,Poured a cloudy golden body with one finger of...,0.813012,4.75,0.9963
241,Zombie Dust,Really grateful to FBarber for hooking me up w...,0.796777,5.00,0.9963


In [17]:
df_sentiment_300.to_csv('taskD.csv', index=None)

### Task E. 

Based on tasks C and D, <b> recommend 3 beers </b> to the customer. Note that in task D, multiple reviews may refer to the same beer. Show the average similarity and sentiment scores for each of the 3 recommended products. 

In [18]:
df_recommend = pd.read_csv('taskD.csv')

In [35]:
df_top3 = df_recommend.groupby(['product_name'])['Sentiment Score','similarity_score','user_rating'].mean().sort_values(by='Sentiment Score',ascending=False).head(3)
df_top3

,Sentiment Score,similarity_score,user_rating
product_name,,,
Trappistes Rochefort 10,0.983233,0.796307,4.50
Abraxas,0.980800,0.796650,4.75
Mother Of All Storms,0.971400,0.803925,4.50


In [20]:
print("The top 3 beers according to the similarity scores are",', '.join(df_top3.index))

The top 3 beers according to the similarity scores are Trappistes Rochefort 10, Abraxas, Mother Of All Storms


### Task F. 
How would your recommendations differ if you ignored the similarity and sentiment scores and simply chose the 3 highest rated beers from your entire dataset? To do this, you need to calculate the average rating for each beer mentioned (remember each beer in your data has multiple reviews, and you need to get the average rating for each beer). Would these three beers meet the requirements of the user looking for recommendations? Why or why not? Justify your answer. Use the user_rating data, similarity and sentiment scores to answer this question. 

In [38]:
df_top3rating = df.groupby(['beer'])['overall'].mean().sort_values(ascending=False).head(3)
df_top3rating

beer
Pliny The Younger                 4.72250
Fou' Foune                        4.66250
Trappist Westvleteren 12 (XII)    4.63625
Name: overall, dtype: float64

In [22]:
print("The top 3 beers according to the user ratings are",', '.join(df_top3rating.index))

The top 3 beers according to the user ratings are Pliny The Younger, Fou' Foune, Trappist Westvleteren 12 (XII)


In [28]:
user_rating = list(df_top3rating.index)

### Top beer recommendation based on sentiment analysis & similarity score (averaging scores from 300 reviews with attribute mentions

In [36]:
df_top3

,Sentiment Score,similarity_score,user_rating
product_name,,,
Trappistes Rochefort 10,0.983233,0.796307,4.50
Abraxas,0.980800,0.796650,4.75
Mother Of All Storms,0.971400,0.803925,4.50


### Top beer solely based on user ratings (Data gotten from averaging overall reviews)

In [37]:
df_top_rating_3 = df_sims_sentiment[df_sims_sentiment['product_name'].isin(user_rating)]
df_top_rating_3.groupby(['product_name'])['Sentiment Score','similarity_score','user_rating'].mean().sort_values(by='user_rating',ascending=False)

,Sentiment Score,similarity_score,user_rating
product_name,,,
Pliny The Younger,0.779790,0.608482,4.72250
Fou' Foune,0.828286,0.713179,4.66250
Trappist Westvleteren 12 (XII),0.820893,0.665318,4.63625


#### The recommendation system is based on these following attributes: crisp, balanced, and fruity which we are assuming the user wants and looks for in a beer. On the other hand, the top three beers above  is based solely on user rating and is not necessarily catered to our user. This can be shown by comparing the similarity scores of the beers that our recommendation system gave and the ones above. The similarity scores of our recommendation system are roughly around 0.80 while the ones shown above are lower by at least 0.10. This is further proved by the sentiment scores. The ones from our recommendation system almost reach a sentiment score of 1.0 which means that the user really enjoys these beers as opposed to the sentiment scores of the top three which are roughly around 0.80. 